# Census Data

The demographics data is sourced from the Census Bureau's Population Estimates Program from 2010 to 2019. The data is collected at the state and county levels. The API links are available here: https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html. The variable descriptions are documented here: https://www.census.gov/data/developers/data-sets/popest-popproj/popest/popest-vars.Vintage_2019.html.

In [1]:
# Import libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
from os import *
import sys
sys.path.append("../")
import urllib.request
import requests
import json

In [15]:
# Change directory
def change_dir(newpath):
    get_path = sys.path[0].split("\\")      
    del get_path[-1]                                                  
    get_path.append(newpath)                    
    path = "\\".join(get_path)              
    os.chdir(path) 

In [2]:
# Source census data
def get_census_data(api_call,fields,level):
    api_query = 'https://api.census.gov/data/2019/pep/' + str(api_call) + '?get=NAME,STATE'
    for field in fields:
        api_query = api_query + "," + field
    api_query = api_query + '&for=' + str(level)
    response = requests.get(api_query)
    formattedResponse = json.loads(response.text)
    return formattedResponse

In [3]:
# Pass a call for demographics data
demographics_api_call = 'charagegroups'
demographics_fields = ['POP','RACE','HISP','SEX','DATE_CODE']
demographics_level = 'county&AGEGROUP=0,30,25,26'
demographics = pd.DataFrame(get_census_data(demographics_api_call, demographics_fields, demographics_level))
demographics.columns = demographics.iloc[0]
demographics = demographics[1:]
demographics

,NAME,STATE,POP,RACE,HISP,SEX,DATE_CODE,AGEGROUP,state,county
1,"Autauga County, Alabama",01,54571,0,0,0,1,0,01,001
2,"Autauga County, Alabama",01,53261,0,1,0,1,0,01,001
3,"Autauga County, Alabama",01,1310,0,2,0,1,0,01,001
4,"Autauga County, Alabama",01,43297,1,0,0,1,0,01,001
5,"Autauga County, Alabama",01,42194,1,1,0,1,0,01,001
...,...,...,...,...,...,...,...,...,...,...
4422596,"Yauco Municipio, Puerto Rico",72,5805,0,0,1,11,30,72,153
4422597,"Yauco Municipio, Puerto Rico",72,5984,0,0,2,11,30,72,153
4422598,"Yauco Municipio, Puerto Rico",72,11681,0,0,0,12,30,72,153
4422599,"Yauco Municipio, Puerto Rico",72,5755,0,0,1,12,30,72,153


In [4]:
#demographics = demographics.astype({'NAME':str,'STATE':int,'POP':int,'RACE':int,'HISP':int,'DATE_CODE':int,'county':int})
demographics.dtypes

0
NAME         object
STATE        object
POP          object
RACE         object
HISP         object
SEX          object
DATE_CODE    object
AGEGROUP     object
state        object
county       object
dtype: object

In [5]:
demographics['NAME'] = demographics['NAME'].astype(str)
demographics['STATE'] = demographics['STATE'].astype(str).astype(int)
demographics['POP'] = demographics['POP'].astype(str).astype(int)
demographics['RACE'] = demographics['RACE'].astype(str).astype(int)
demographics['HISP'] = demographics['HISP'].astype(str).astype(int)
demographics['SEX'] = demographics['SEX'].astype(str).astype(int)
demographics['DATE_CODE'] = demographics['DATE_CODE'].astype(str).astype(int)
demographics['state'] = demographics['state'].astype(str).astype(int)
demographics['county'] = demographics['county'].astype(str).astype(int)
demographics['AGEGROUP'] = demographics['AGEGROUP'].astype(str).astype(int)
demographics.dtypes

0
NAME         object
STATE         int32
POP           int32
RACE          int32
HISP          int32
SEX           int32
DATE_CODE     int32
AGEGROUP      int32
state         int32
county        int32
dtype: object

In [6]:
demographics = demographics[demographics['STATE']<57]
demographics = demographics[demographics['DATE_CODE']>7]
demographics = demographics[demographics['RACE']<7]
demographics

,NAME,STATE,POP,RACE,HISP,SEX,DATE_CODE,AGEGROUP,state,county
757,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1
758,"Autauga County, Alabama",1,53391,0,1,0,8,0,1,1
759,"Autauga County, Alabama",1,1473,0,2,0,8,0,1,1
760,"Autauga County, Alabama",1,42650,1,0,0,8,0,1,1
761,"Autauga County, Alabama",1,41448,1,1,0,8,0,1,1
...,...,...,...,...,...,...,...,...,...,...
4419788,"Weston County, Wyoming",56,1275,0,0,1,11,30,56,45
4419789,"Weston County, Wyoming",56,1061,0,0,2,11,30,56,45
4419790,"Weston County, Wyoming",56,2363,0,0,0,12,30,56,45
4419791,"Weston County, Wyoming",56,1296,0,0,1,12,30,56,45


In [7]:
# Pass a call for migration data
migration_api_call = 'components'
migration_fields = ['NETMIG','DOMESTICMIG','INTERNATIONALMIG','PERIOD_CODE']
migration_level = 'county'
migration = pd.DataFrame(get_census_data(migration_api_call, migration_fields, migration_level))
migration.columns = migration.iloc[0]
migration = migration[1:]
migration

,NAME,STATE,NETMIG,DOMESTICMIG,INTERNATIONALMIG,PERIOD_CODE,state,county
1,"Cape Girardeau County, Missouri",29,134,115,19,1,29,031
2,"Cape Girardeau County, Missouri",29,598,458,140,2,29,031
3,"Cape Girardeau County, Missouri",29,231,117,114,3,29,031
4,"Cape Girardeau County, Missouri",29,242,151,91,4,29,031
5,"Cape Girardeau County, Missouri",29,287,158,129,5,29,031
...,...,...,...,...,...,...,...,...
31416,"Murray County, Georgia",13,80,12,68,6,13,213
31417,"Murray County, Georgia",13,-189,-250,61,7,13,213
31418,"Murray County, Georgia",13,344,317,27,8,13,213
31419,"Murray County, Georgia",13,31,10,21,9,13,213


In [8]:
migration = migration.astype({'NAME':str,'STATE':int,'NETMIG':int,'DOMESTICMIG':int,'INTERNATIONALMIG':int,'PERIOD_CODE':int,'state':int,'county':int})
migration.dtypes

0
NAME                object
STATE                int32
NETMIG               int32
DOMESTICMIG          int32
INTERNATIONALMIG     int32
PERIOD_CODE          int32
state                int32
county               int32
dtype: object

In [9]:
merged = demographics.merge(migration)
merged

,NAME,STATE,POP,RACE,HISP,SEX,DATE_CODE,AGEGROUP,state,county,NETMIG,DOMESTICMIG,INTERNATIONALMIG,PERIOD_CODE
0,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1,172,147,25,1
1,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1,331,327,4,2
2,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1,-343,-329,-14,3
3,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1,-214,-226,12,4
4,"Autauga County, Alabama",1,54864,0,0,0,8,0,1,1,108,101,7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11311195,"Weston County, Wyoming",56,1067,0,0,2,12,30,56,45,66,24,42,6
11311196,"Weston County, Wyoming",56,1067,0,0,2,12,30,56,45,5,-9,14,7
11311197,"Weston County, Wyoming",56,1067,0,0,2,12,30,56,45,-257,-275,18,8
11311198,"Weston County, Wyoming",56,1067,0,0,2,12,30,56,45,-48,-65,17,9


In [10]:
# Change directory
def change_dir(newpath):
    get_path = sys.path[0].split("\\")      
    del get_path[-1]                                                  
    get_path.append(newpath)                    
    path = "\\".join(get_path)              
    os.chdir(path) 

In [11]:
merged['state'] = merged['state'].astype(int)
merged['county'] = merged['county'].astype(int)

In [12]:
change_dir('input')
add_CBSA_codes = pd.read_csv('delineation_files.csv')
add_CBSA_codes = add_CBSA_codes.rename(columns={"FIPS State Code": "state", "FIPS County Code": "county"})
add_CBSA_codes['state'] = add_CBSA_codes['state'].astype(int)
add_CBSA_codes['county'] = add_CBSA_codes['county'].astype(int)

In [13]:
merged = merged.merge(add_CBSA_codes)

In [14]:
change_dir('output')
merged.to_csv('demographics.csv')

In [15]:
merged['CBSA Code'].nunique()

984

## Demographics for stage 2 analysis

In [131]:
# Get demographics data
api_query = "https://api.census.gov/data/2019/pep/charagegroups?get=POP,NAME,RACE,HISP,STATE&for=county&DATE_CODE=12"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
dg = pd.DataFrame(formattedResponse)
dg.columns = dg.iloc[0]
dg = dg.iloc[1:,:]
dg[['POP','RACE','HISP','STATE','state','county']] = dg[['POP','RACE','HISP','STATE','state','county']].astype(int)
dg = dg[dg['RACE']<7]
dg

,POP,NAME,RACE,HISP,STATE,DATE_CODE,state,county
1,22124,"Coahoma County, Mississippi",0,0,28,12,28,27
2,21757,"Coahoma County, Mississippi",0,1,28,12,28,27
3,367,"Coahoma County, Mississippi",0,2,28,12,28,27
4,4626,"Coahoma County, Mississippi",1,0,28,12,28,27
5,4412,"Coahoma County, Mississippi",1,1,28,12,28,27
...,...,...,...,...,...,...,...,...
113171,0,"Stanton County, Kansas",5,1,20,12,20,187
113172,4,"Stanton County, Kansas",5,2,20,12,20,187
113173,72,"Stanton County, Kansas",6,0,20,12,20,187
113174,61,"Stanton County, Kansas",6,1,20,12,20,187


In [138]:
# Get age group data
api_query = "https://api.census.gov/data/2019/pep/charagegroups?get=POP,NAME,AGEGROUP&for=county&DATE_CODE=12"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
age = pd.DataFrame(formattedResponse)
age.columns = age.iloc[0]
age = age.iloc[1:,:]
age[['state','county']] = age[['state','county']].astype(int)
age = age[age['AGEGROUP']=='26']
age

,POP,NAME,AGEGROUP,DATE_CODE,state,county
27,3541,"Coahoma County, Mississippi",26,12,28,27
59,3399,"Jasper County, Mississippi",26,12,28,61
91,11827,"Jones County, Mississippi",26,12,28,67
123,2846,"Walthall County, Mississippi",26,12,28,147
155,6954,"Monroe County, Mississippi",26,12,28,95
...,...,...,...,...,...,...
102905,28886,"Scott County, Iowa",26,12,19,163
102937,11351,"Dallas County, Iowa",26,12,19,49
102969,2840,"O'Brien County, Iowa",26,12,19,141
103001,1152,"Sherman County, Kansas",26,12,20,181


In [140]:
# Merge age group with the demographics data
add_age_group = dg.merge(age, on=['NAME','state','county'])
add_age_group = add_age_group.rename(columns={'POP_x':'population','POP_y':'above_65'})
add_age_group = add_age_group.drop(columns=['AGEGROUP','DATE_CODE'])
add_age_group

,population,NAME,RACE,HISP,STATE,state,county,above_65,POP
0,22124,"Coahoma County, Mississippi",0,0,28,28,27,3541,3541
1,21757,"Coahoma County, Mississippi",0,1,28,28,27,3541,3541
2,367,"Coahoma County, Mississippi",0,2,28,28,27,3541,3541
3,4626,"Coahoma County, Mississippi",1,0,28,28,27,3541,3541
4,4412,"Coahoma County, Mississippi",1,1,28,28,27,3541,3541
...,...,...,...,...,...,...,...,...,...
66055,0,"Stanton County, Kansas",5,1,20,20,187,347,347
66056,4,"Stanton County, Kansas",5,2,20,20,187,347,347
66057,72,"Stanton County, Kansas",6,0,20,20,187,347,347
66058,61,"Stanton County, Kansas",6,1,20,20,187,347,347


In [151]:
# Merge the CBSA crosswalk
dg = add_age_group
change_dir('input')
add_CBSA_codes = pd.read_csv('delineation_files.csv')
add_CBSA_codes = add_CBSA_codes.rename(columns={"FIPS State Code": "state", "FIPS County Code": "county"})
add_CBSA_codes[['state','county']] = add_CBSA_codes[['state','county']].astype(int)
dg = dg.merge(add_CBSA_codes)
dg

,population,NAME,RACE,HISP,STATE,state,county,above_65,POP,CBSA Code,CBSA Title,County/County Equivalent,State Name
0,22124,"Coahoma County, Mississippi",0,0,28,28,27,3541,3541,17260,"Clarksdale, MS",Coahoma County,Mississippi
1,21757,"Coahoma County, Mississippi",0,1,28,28,27,3541,3541,17260,"Clarksdale, MS",Coahoma County,Mississippi
2,367,"Coahoma County, Mississippi",0,2,28,28,27,3541,3541,17260,"Clarksdale, MS",Coahoma County,Mississippi
3,4626,"Coahoma County, Mississippi",1,0,28,28,27,3541,3541,17260,"Clarksdale, MS",Coahoma County,Mississippi
4,4412,"Coahoma County, Mississippi",1,1,28,28,27,3541,3541,17260,"Clarksdale, MS",Coahoma County,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56349,51,"Dallas County, Iowa",5,1,19,19,49,11351,11351,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
56350,14,"Dallas County, Iowa",5,2,19,19,49,11351,11351,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
56351,1571,"Dallas County, Iowa",6,0,19,19,49,11351,11351,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
56352,1358,"Dallas County, Iowa",6,1,19,19,49,11351,11351,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa


In [1]:
# Get education data - ACS 5 year estimates
api_query = "https://api.census.gov/data/2019/acs/acs5/cprofile?get=NAME,CP02_2019_068E,CP02_2019_063E,CP02_2019_064E,CP02_2019_062E&for=county"
response = requests.get(api_query)
formattedResponse = json.loads(response.text)
df = pd.DataFrame(formattedResponse)
df.columns = df.iloc[0]
df = df.iloc[1:,:]
df = df.rename(columns={'CP02_2019_065E':'bachelors','CP02_2019_063E':'college_no_degree','CP02_2019_064E':'associates','CP02_2019_062E':'ged'})
df[['bachelors','college_no_degree','associates','ged']] = df[['bachelors','college_no_degree','associates','ged']].astype(float)
df[['state','county']] = df[['state','county']].astype(int)
df

NameError: name 'requests' is not defined

In [177]:
#m1 = df.merge(dg, on=['county','state'],how='left')
m1 = pd.merge(dg,df,left_on=['county','state'], how='left').drop_duplicates()
m1 = m1.drop(columns=['NAME_y','STATE','County/County Equivalent','State Name','CBSA Code','CBSA Title'])
m1

,population,NAME_x,RACE,HISP,state,county,above_65,POP,bachelors,college_no_degree,associates,ged
0,22124,"Coahoma County, Mississippi",0,0,28,27,3541,3541,10.4,24.5,13.3,25.2
1,21757,"Coahoma County, Mississippi",0,1,28,27,3541,3541,10.4,24.5,13.3,25.2
2,367,"Coahoma County, Mississippi",0,2,28,27,3541,3541,10.4,24.5,13.3,25.2
3,4626,"Coahoma County, Mississippi",1,0,28,27,3541,3541,10.4,24.5,13.3,25.2
4,4412,"Coahoma County, Mississippi",1,1,28,27,3541,3541,10.4,24.5,13.3,25.2
...,...,...,...,...,...,...,...,...,...,...,...,...
56349,51,"Dallas County, Iowa",5,1,19,49,11351,11351,35.0,16.9,10.6,17.7
56350,14,"Dallas County, Iowa",5,2,19,49,11351,11351,35.0,16.9,10.6,17.7
56351,1571,"Dallas County, Iowa",6,0,19,49,11351,11351,35.0,16.9,10.6,17.7
56352,1358,"Dallas County, Iowa",6,1,19,49,11351,11351,35.0,16.9,10.6,17.7


In [173]:
m2 = pd.merge(m1,add_CBSA_codes,on=['county','state'], how='left').drop_duplicates()
m2

,population,NAME_x,RACE,HISP,state,county,above_65,POP,bachelors,college_no_degree,associates,ged,CBSA Code,CBSA Title,County/County Equivalent,State Name
0,22124,"Coahoma County, Mississippi",0,0,28,27,3541,3541,10.4,24.5,13.3,25.2,17260,"Clarksdale, MS",Coahoma County,Mississippi
1,21757,"Coahoma County, Mississippi",0,1,28,27,3541,3541,10.4,24.5,13.3,25.2,17260,"Clarksdale, MS",Coahoma County,Mississippi
2,367,"Coahoma County, Mississippi",0,2,28,27,3541,3541,10.4,24.5,13.3,25.2,17260,"Clarksdale, MS",Coahoma County,Mississippi
3,4626,"Coahoma County, Mississippi",1,0,28,27,3541,3541,10.4,24.5,13.3,25.2,17260,"Clarksdale, MS",Coahoma County,Mississippi
4,4412,"Coahoma County, Mississippi",1,1,28,27,3541,3541,10.4,24.5,13.3,25.2,17260,"Clarksdale, MS",Coahoma County,Mississippi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533343,51,"Dallas County, Iowa",5,1,19,49,11351,11351,35.0,16.9,10.6,17.7,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
533344,14,"Dallas County, Iowa",5,2,19,49,11351,11351,35.0,16.9,10.6,17.7,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
533345,1571,"Dallas County, Iowa",6,0,19,49,11351,11351,35.0,16.9,10.6,17.7,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa
533346,1358,"Dallas County, Iowa",6,1,19,49,11351,11351,35.0,16.9,10.6,17.7,19780,"Des Moines-West Des Moines, IA",Dallas County,Iowa


In [174]:
m2['CBSA Code'].nunique()

996

In [176]:
change_dir('output')
m1.to_csv('demographics.csv',index=False)